# Tutorial #1: Interfacing w/ the Library

**Target Audience:** Design Engineers / Front-end User

**Learning Objectives:**
1. Primary Functionality
    1. [Selecting a component](#selecting)
    2. [Meaning / notation of characteristics](#reading-characteristics)
    3. [Finding geometrical parameters which best suits target characteristics](#select-characteristics)
    4. [Export to `.gds` or dictionary compatable with `QComponent(options=...)`](#export)
2. Secondary Functionality
    1. Seeing who has contributed what data
    2. Seeing what was the simulation method of the contributed data


# Section 1: Primary Functionality

## Section 1.1: Selecting a Component <a id="selecting"></a>

First, let's see which components are currently supported.

In [1]:
import metal_library

print(metal_library.supported_components)

['TransmonCross', 'TransmonPocket']


Now we can place one of the supported components into `Reader`. This is a class meant to parse the library (located at `C:\\...\metal_library\library`)

In [2]:
from metal_library import Reader

reader = Reader(component_name='TransmonCross')
print(reader)

Interfacing with the library will be conducted through this object.

## Section 1.2: Understanding Structure of Characteristics <a id="reading-characteristics"></a>

Since we're conducting science, we must use exact language when talking about the characteristics we're pulling.

Example of **imprecise** language:
- 👎 "Get the coupling strength?"

Example of **precise** language:
- 👍 "Get the coupling strength between the readout cavity and qubit"
- 👍 "Get the coupling strength between qubit #1 and the bus (connecting qubit #1 and qubit #2)"

The following attributes / functions were created to help the front-end user follow precisely what each component characteristic means.



### Component Types
In simulations, we don't always simulate the entire chip at once. So there's a property `component_types` to view which combinations we've simulated.

It also includes a little blurb to help clarify what the names imply!

In [3]:
reader.component_types

╒═════════════╤════════════════════════════════════════════════╤════════════════════════════════════════════╕
│ QubitOnly   │ QubitCavity                                    │ QubitDriveline                             │
╞═════════════╪════════════════════════════════════════════════╪════════════════════════════════════════════╡
│ Qubit only  │ Qubit capacitvely coupled to a readout cavity. │ Qubit capacitvely coupled to a drive line. │
╘═════════════╧════════════════════════════════════════════════╧════════════════════════════════════════════╛


['QubitOnly', 'QubitCavity', 'QubitDriveline']

### Characteristics of Component Type

Within each `component_type`, the types of observables / component characteristics you can extract change. To see which characteristics are supported, we can use `.get_characteristic_info`.

In [4]:
reader.get_characteristic_info(component_type="QubitOnly");

╒═════════════════════════╤═══════════════════════════════════════╤═════════╤═══════════════╕
│ CSV Column Name         │ Description                           │ Units   │ Math Symbol   │
╞═════════════════════════╪═══════════════════════════════════════╪═════════╪═══════════════╡
│ Qubit_Frequency_GHz     │ Qubit 01 Transition Freq (Linear GHz) │ GHz     │ f_{q,01}      │
├─────────────────────────┼───────────────────────────────────────┼─────────┼───────────────┤
│ Qubit_Anharmonicity_MHz │ Qubit Anharmonicity (Linear MHz)      │ MHz     │ \alpha_{q}    │
╘═════════════════════════╧═══════════════════════════════════════╧═════════╧═══════════════╛


In [5]:
reader.get_characteristic_info(component_type="QubitCavity");

╒═════════════════════════╤════════════════════════════════════════════════╤═════════╤═══════════════╕
│ CSV Column Name         │ Description                                    │ Units   │ Math Symbol   │
╞═════════════════════════╪════════════════════════════════════════════════╪═════════╪═══════════════╡
│ Qubit_Frequency_GHz     │ Qubit 01 Transition Freq (Linear GHz)          │ GHz     │ f_{q,01}      │
├─────────────────────────┼────────────────────────────────────────────────┼─────────┼───────────────┤
│ Qubit_Anharmonicity_MHz │ Qubit Anharmonicity (Linear MHz)               │ MHz     │ \alpha_{q}    │
├─────────────────────────┼────────────────────────────────────────────────┼─────────┼───────────────┤
│ Cavity_Frequency_GHz    │ Cavity Resonant Freq (Linear MHz)              │ GHz     │ _{c}          │
├─────────────────────────┼────────────────────────────────────────────────┼─────────┼───────────────┤
│ Coupling_Strength_MHz   │ Coupling between qubit and cavity (Linear MHz

In [6]:
reader.get_characteristic_info(component_type="QubitDriveline");

╒══════════════════════════╤═════════════════════════════════════════════╤═════════╤═══════════════╕
│ CSV Column Name          │ Description                                 │ Units   │ Math Symbol   │
╞══════════════════════════╪═════════════════════════════════════════════╪═════════╪═══════════════╡
│ Qubit_Frequency_GHz      │ Qubit 01 Transition Freq (Linear GHz)       │ GHz     │ f_{q,01}      │
├──────────────────────────┼─────────────────────────────────────────────┼─────────┼───────────────┤
│ Qubit_Anharmonicity_MHz  │ Qubit Anharmonicity (Linear MHz)            │ MHz     │ \alpha_{q}    │
├──────────────────────────┼─────────────────────────────────────────────┼─────────┼───────────────┤
│ Driveline_Decay_Rate_MHz │ Feedline limited T1 Decay Rate (linear MHz) │ MHz     │ \kappa        │
╘══════════════════════════╧═════════════════════════════════════════════╧═════════╧═══════════════╛


## Section 1.3: Finding a Geometry for Given Characteristics

In [7]:
from metal_library import Selector

In [9]:
reader.read_library(component_type='QubitOnly')
reader.library

ERROR: There are no columns in your `.csv`. This error probably came from using QLibrarian.append_csv() to make a new file.
                     Data won't be formatted properly. 


{}

In [12]:
import os
import pandas as pd

csv_file_name = "QubitOnly.csv"

component_type_path = os.path.join(reader.path, csv_file_name)
df = pd.read_csv(component_type_path)
df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,28,29,30,31,32,33,34,35,36,37
0,0,0,0.0um,0,main,1,0,30um,5um,10um,...,0,0,185um,4um,5.1um,10um,NaN,4368.966835,208.731884,{'project_info': {'pinfo': design_name ...
1,1,0,0.0um,0,main,1,0,30um,5um,10um,...,0,0,185um,4um,5.1um,10um,NaN,4368.966835,208.731884,{'project_info': {'pinfo': design_name ...
2,2,0,0.0um,0,main,1,0,30um,5um,10um,...,0,0,185um,4um,5.1um,10um,NaN,4368.966835,208.731884,{'project_info': {'pinfo': design_name ...
3,3,0,0.0um,0,main,1,0,30um,5um,10um,...,0,0,185um,4um,5.1um,10um,NaN,4193.934789,210.539654,{'project_info': {'pinfo': design_name ...
4,4,0,0.0um,0,main,1,0,30um,5um,10um,...,0,0,185um,4um,5.1um,10um,NaN,4031.690829,211.554005,{'project_info': {'pinfo': design_name ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
723,723,0,0.0um,0,main,1,0,30um,5um,10um,...,0,0,215um,10um,5.1um,15um,NaN,3799.752357,168.592842,{'project_info': {'pinfo': design_name ...
724,724,0,0.0um,0,main,1,0,30um,5um,10um,...,0,0,215um,10um,5.1um,15um,NaN,3654.123957,169.409177,{'project_info': {'pinfo': design_name ...
725,725,0,0.0um,0,main,1,0,30um,5um,10um,...,0,0,215um,10um,5.1um,15um,NaN,3875.563528,159.563999,{'project_info': {'pinfo': design_name ...
726,726,0,0.0um,0,main,1,0,30um,5um,10um,...,0,0,215um,10um,5.1um,15um,NaN,3713.782376,160.186036,{'project_info': {'pinfo': design_name ...
